In [1]:
import os


In [2]:
%pwd

'c:\\Users\\azfar\\Downloads\\wine-ml-proj\\wine-ml\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\azfar\\Downloads\\wine-ml-proj\\wine-ml'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [7]:
from wineml.constants import *
from wineml.utils.common import read_yaml, create_directories

In [8]:
from wineml.entity.config_entity import DataValidationConfig
from pathlib import Path
from wineml.utils.common import read_yaml, create_directories
from wineml.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from box import ConfigBox


class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
        schema_filepath: Path = SCHEMA_FILE_PATH
    ):
        self.config: ConfigBox = read_yaml(config_filepath)
        self.params: ConfigBox = read_yaml(params_filepath)

        # ✅ Optional schema load
        self.schema: ConfigBox | None = None
        if schema_filepath.exists():
            self.schema = read_yaml(schema_filepath)

        create_directories([Path(self.config.artifacts_root)])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS if self.schema else {}

        create_directories([Path(config.root_dir)])

        data_validation_config = DataValidationConfig(
            root_dir=Path(config.root_dir),
            STATUS_FILE=Path(config.STATUS_FILE),
            unzip_data_dir=Path(config.unzip_data_dir),
            all_schema=schema  # schema is now a dict
        )
        return data_validation_config

In [9]:
import os
from wineml.logging.logger import logging
import pandas as pd

In [16]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    
    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [14]:
%pwd


'c:\\Users\\azfar\\Downloads\\wine-ml-proj\\wine-ml'

In [18]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-05-06 23:47:04,087]: INFO: common: yaml file: C:\Users\azfar\Downloads\wine-ml-proj\wine-ml\config\config.yaml loaded successfully]
[2025-05-06 23:47:04,090]: INFO: common: yaml file: C:\Users\azfar\Downloads\wine-ml-proj\wine-ml\params.yaml loaded successfully]
[2025-05-06 23:47:04,094]: INFO: common: created directory at: artifacts]
[2025-05-06 23:47:04,096]: INFO: common: created directory at: artifacts\data_validation]
